<a href="https://colab.research.google.com/github/deepa22-eng/Machine-Learning-2026/blob/main/Assignment_2_Deepa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chemical Applications of Machine Learning (CHEM 4930/5610) - Spring 2026

### Assignment 2 - Deadline 2/3/2026
Points 10

#### General Comments
All figures and graph should have approriate labels on the two axis, and should include a legend with appropriate labels of the different plots.

The notebook should be return in working format. That is, I should be able to reset all the output and re-run all the cells and get the same results as you obtained.

**You should start by saving a copy of the notebook to your Google Drive so you preserve all changes.**

**Please add your name as a suffix to the filname**

**Student Name**: Deepa Ranabhat

**AI usage statement:**
Used ChatGPT basically for generating the python code and understand them how to implement them for the data analysis

### Task 1 - 10 points

In this task, we will consider the Bradley Melting Point Dataset, which is curated chemical dataset with melting points of around 3,000 chemical compounds, see [here](https://www.kaggle.com/datasets/aliffaagnur/melting-point-chemical-dataset/data).

This dataset is stored in a comma-separated values (csv) file, which is common format used to start data in text files. We can load this into a pandas DataFrame using the `load_csv` function.

In this dataset, we have the compounds names, SMILES strings, and the melting point in Celsius.

#### A)
Identify in the dataset the chemical compounds with the 5 lowest melting points and 5 highest melting points and visualize their 2D chemical structure using RDKit and the [mols2grid package](https://mols2grid.readthedocs.io/en/latest/), where you display the melting point values on the grid, see [here](https://colab.research.google.com/github/PatWalters/practical_cheminformatics_tutorials/blob/main/fundamentals/A_Whirlwind_Introduction_To_The_RDKit.ipynb#scrollTo=N3CR7rMF3sg7) for an example of the usage of mols2grid.

#### B)
Calculate the following properties for the molecules using RDKIt:
- The molecular weight
- The number of heavy atoms
- Number of hydrogen bond acceptors
- Number of hydrogen bond donors
- [Octanol-water partition coefficient - LogP](https://pubs-acs-org.libproxy.library.unt.edu/doi/10.1021/ci990307l)
- [Topological polar surface area (TPSA) descriptor](https://pubs-acs-org.libproxy.library.unt.edu/doi/abs/10.1021/jm000942e)
- Topological polar surface area (TPSA) descriptor, including S and P atoms, see [here](https://www.rdkit.org/docs/RDKit_Book.html#implementation-of-the-tpsa-descriptor)

Note: for some of the molecules, the TPSA descriptor will give a value of zero. When doing any analysis for the TPSA descriptor, you should ignore these values.

#### C)
Write out to a new csv file values of all the properties calculated in B) along with the compound names, SMILES strings, and the melting point in Celsius. Here, when writing this file, you should ignore any compounds where the SMILES conversion did not work correctly.

#### D)
Perform a linear regression analysis using scikit-learn where you look at the correlation of each of the properties calculated in B) with melting temperature. Here, each property should be considered individually.

To avoid outliers, filter out (i.e., remove) the compounds with the lowest 10% and the highest 10% melting temperature. Make a histogram that shows this filtering. Furthermore, for each property, filter out the compounds with lowest 10% and highest 10% values (again making a histogram that shows this filtering). Only consider the joint remaining compounds in your linear regression analysis for each property.

When performing the linear regression, employ a 70%/30% training/test split.

Calculate the coefficient of determination, $R^2$, for both the training dataset and the test dataset and report both.

You should make figure that shows the data along with the linear curve coming from the linear regression. In the figure, it should be clear which data points are in the training and test set (e.g., by having them in different colors). Include the $R^2$ values on the figure.

From your analysis, which of the properties correlates best with the melting temperature?

#### E)
For two of the properties from D) (e.g., the ones that correlate best with the melting point), perform [RANSAC](https://en.wikipedia.org/wiki/Random_sample_consensus) regression, which is method that takes outliers into account when performing linear regression and does not include them in the final modeling, see [here](https://scikit-learn.org/stable/auto_examples/linear_model/plot_ransac.html).

In the figure, it should be clear which data points are in inlier set and which are in the outlier set (e.g., by showing them in different colors).


In [ ]:
# Bash script to download all the dataset. Don't worry if you don't understand it
%%bash

url="https://raw.githubusercontent.com/valsson-group/UNT-ChemicalApplicationsOfMachineLearning-Spring2026/refs/heads/main/Assignment-2/"
dataset_filename="BradleyDoublePlusGoodMeltingPointDataset.csv"

rm -f ${dataset_filename}

wget ${url}/${dataset_filename} &> /dev/null

ls

A)  Chemical compounds with the 5 lowest melting points and 5 highest melting points and visualize their 2D chemical structure using RDKit and the mols2grid package

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("BradleyDoublePlusGoodMeltingPointDataset.csv")

In [ ]:
#print (data_mp.keys)
print(list(data.keys()))

In [ ]:
data["mpC"]

In [ ]:
melting_point_C= data ["mpC"]
print (melting_point_C)

In [ ]:
lowest_5_molecules = np.argpartition(melting_point_C,5)[:5]
print (lowest_5_molecules)

In [ ]:
data[['name', 'smiles', 'mpC']].iloc[lowest_5_molecules]

In [ ]:
highest_5_molecules = np.argpartition(melting_point_C,-5)[-5:]
print (highest_5_molecules)

In [ ]:
data[['name', 'smiles', 'mpC']].iloc[highest_5_molecules]

In [ ]:
# the %%capture command will surpress output to screen
%%capture
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install rdkit mols2grid

In [ ]:

import mols2grid


In [ ]:
# by passing the dataframe, and giving the column with the SMILES string, we can
# plot all molecules
# we can also add information to the figures by using the subset variable
mols2grid.display(data,smiles_col='smiles',subset=['img','name','smiles','mpC'])

In [ ]:
# by passing the dataframe, and giving the column with the SMILES string, we can
# plot all molecules
# we can also add information to the figures by using the subset variable
# Here we also format the string for the melting point to show the C.

def mp_str(x):
    return f'{x:.2f} C'

mols2grid.display(data.iloc[lowest_5_molecules],smiles_col='smiles',subset=['img','name','mpC'], transform={"mpC": mp_str})

In [ ]:

def mp_str(x):
    return f'{x:.2f} C'

mols2grid.display(data.iloc[highest_5_molecules],smiles_col='smiles',subset=['img','name','mpC'], transform={"mpC": mp_str})

B)
Calculate the following properties for the molecules using RDKIt:

The molecular weight
The number of heavy atoms
Number of hydrogen bond acceptors
Number of hydrogen bond donors
Octanol-water partition coefficient - LogP
Topological polar surface area (TPSA) descriptor
Topological polar surface area (TPSA) descriptor, including S and P atoms

In [ ]:
from rdkit.Chem import Descriptors, rdMolDescriptors


In [ ]:
print("Descriptors.__")
for des in Descriptors._descList: print("-",des[0])

In [ ]:
# This function calculates the MolWt', 'HeavyAtomCount', 'LogP', 'HBA', 'HBD', 'TPSA', 'TPSA_S_P'
# from a SMILES string using RDKit. If the SMILES is invalid,
# it returns NaN instead of causing an error.

from rdkit import Chem
from rdkit.Chem import Descriptors

def molecular_weight(smi):
  mol = Chem.MolFromSmiles(smi)
  if mol is not None:
    return Descriptors.MolWt(mol)
  else:
    return np.nan

def Heavy_atom_count(smi):
  mol = Chem.MolFromSmiles(smi)
  if mol is not None:
    return Descriptors.HeavyAtomCount(mol)
  else:
    return np.nan

def mol_logP(smi):
  mol = Chem.MolFromSmiles(smi)
  if mol is not None:
    return Descriptors.MolLogP(mol)
  else:
    return np.nan

def hba (smi):
  mol = Chem.MolFromSmiles(smi)
  if mol is not None:
    return rdMolDescriptors.CalcNumHBA(mol)
  else:
    return np.nan

def hbd (smi):
  mol = Chem.MolFromSmiles(smi)
  if mol is not None:
    return rdMolDescriptors.CalcNumHBD(mol)
  else:
    return np.nan

def tpsa (smi):
  mol = Chem.MolFromSmiles(smi)
  if mol is not None:
    return rdMolDescriptors.CalcTPSA(mol)
  else:
    return np.nan


def tpsa_sp(smi):
  mol = Chem.MolFromSmiles(smi)
  if mol is not None:
    return rdMolDescriptors.CalcTPSA(mol, includeSandP= True)
  else:
    return np.nan




In [ ]:
# here we calculate some property and add that to the dataframe
data['MolWt'] = [molecular_weight(smi) for smi in data['smiles']]
data['HeavyAtomCount'] = [Heavy_atom_count(smi) for smi in data['smiles']]
data['LogP'] = [mol_logP(smi) for smi in data['smiles']]
data['HBA'] = [hba(smi) for smi in data['smiles']]
data['HBD'] = [hbd(smi) for smi in data['smiles']]
data['TPSA'] = [tpsa(smi) for smi in data['smiles']]
data['TPSA_S_P'] = [tpsa_sp(smi)for smi in data['smiles']]

In [ ]:
data.describe()

In [ ]:
# Drop rows where any key molecular descriptor is missing (NaN).
# This ensures all molecules used in regression/plots have complete data.

data_ignore_nan = data.dropna(
    subset=['MolWt', 'HeavyAtomCount', 'LogP', 'HBA', 'HBD', 'TPSA', 'TPSA_S_P']
)


In [ ]:
data_ignore_nan

C) New csv file values of all the properties calculated in B) along with the compound names, SMILES strings, and the melting point in Celsius.

In [ ]:
data_ignore_nan.to_csv("test_with_descriptors.csv", index=False)


In [ ]:
!head test_with_descriptors.csv

In [ ]:
data_ignore_nan[[ 'key', 'name', 'MolWt', 'HeavyAtomCount', 'LogP', 'HBA', 'HBD', 'TPSA', 'TPSA_S_P'] ].to_csv("test-subset.csv",index=False)

In [ ]:
!head test-subset.csv


D)  Linear regression analysis using scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_clean=pd.read_csv ("test_with_descriptors.csv")

In [ ]:
print(list(data.keys()))

In [ ]:
data_clean

Melting Point

In [ ]:
mp_values = data_clean["mpC"].values

In [ ]:
# here we use the np.percentile function to find 10-th percentile and 90-th percentile of the dataset

values_percentile_10 = np.percentile(mp_values,10)
values_percentile_90 = np.percentile(mp_values,90)


print(values_percentile_10)
print(values_percentile_90)

In [ ]:
 #we now create a numpy logical mask to select the elements that are
# within 10% of 90% of the data (that is excluding the lowest 10% and
# highest 10% values of the data)
#
# (values > values_percentile_10) selects values that are higher than values_percentile_10
# (values < values_percentile_90) selects values that are lower than values_percentile_90
# the "&" is a boolean AND operator to select elements that fulfil both.
mask_10_to_90 = (mp_values > values_percentile_10) & (mp_values < values_percentile_90)

values_filtered = mp_values[mask_10_to_90]
# the "~" is a boolean NOT operator
values_filtered_out = mp_values[~mask_10_to_90]

print("values.size:",mp_values.size)
print("values_filtered.size",values_filtered.size)
print("values_filtered_out",values_filtered_out.size)

In [ ]:
# plot the data, this will not look correctly as we only
# pass the values to the plot function so that the
# x-axis is in the index of the element
plt.plot(values_filtered,'.',label="filtered")
plt.plot(values_filtered_out,'.',label="filtered out")
plt.xlabel("Index")
plt.ylabel("Values")
plt.legend()

In [ ]:
# make plots that shows the filtering

plt.hist(mp_values,bins=100,label="Full set",density=True, alpha=0.5)
plt.hist(values_filtered,bins=50,label="Filtered set",density=True, alpha=0.5)
plt.xlabel("value")
plt.ylabel("frequency")
# This is to plot vertical lines for the boundary at 10% and 90%
# using axvline. Here we will need to use the plt.gca() to get the
# axis object for the plot
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
#plt.show()
plt.legend()

# note that the KDE plot will "leak" outside the boundry
# lines as the KDE employs finite width Gaussian kernels
sns.kdeplot(mp_values,label="Full set")
sns.kdeplot(values_filtered,label="Filtered set")
plt.xlabel("value")
plt.ylabel("density")
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
plt.legend()
#plt.show()

MolWt

In [ ]:
molwt_values = data_clean["MolWt"].values

In [ ]:
values_percentile_10 = np.percentile(molwt_values,10)
values_percentile_90 = np.percentile(molwt_values,90)

print(values_percentile_10)
print(values_percentile_90)

In [ ]:
mask_10_to_90 = (molwt_values > values_percentile_10) & (molwt_values < values_percentile_90)

values_filtered = molwt_values[mask_10_to_90]
# the "~" is a boolean NOT operator
values_filtered_out = molwt_values[~mask_10_to_90]

#For regression (molwt + mpC together)
data_molwt_filtered = data_clean[mask_10_to_90]

print("values.size:",molwt_values.size)
print("values_filtered.size",values_filtered.size)
print("values_filtered_out",values_filtered_out.size)

In [ ]:
plt.plot(values_filtered,'.',label="filtered")
plt.plot(values_filtered_out,'.',label="filtered out")
plt.xlabel("Index")
plt.ylabel("MolWt")
plt.legend()

In [ ]:
# make plots that shows the filtering

plt.hist(molwt_values,bins=100,label="Full set",density=True, alpha=0.5)
plt.hist(values_filtered,bins=50,label="Filtered set",density=True, alpha=0.5)
plt.xlabel("Molwt")
plt.ylabel("frequency")
# This is to plot vertical lines for the boundary at 10% and 90%
# using axvline. Here we will need to use the plt.gca() to get the
# axis object for the plot
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
#plt.show()
plt.legend()

# note that the KDE plot will "leak" outside the boundry
# lines as the KDE employs finite width Gaussian kernels
sns.kdeplot(molwt_values,label="Full set")
sns.kdeplot(values_filtered,label="Filtered set")
plt.xlabel("Molwt")
plt.ylabel("density")
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
plt.legend()
#plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
# Linear regression
X = data_molwt_filtered[['MolWt']].values
y = data_molwt_filtered['mpC'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
r2_train = r2_score(y_train, model.predict(X_train))
r2_test = r2_score(y_test, model.predict(X_test))
print(f"MolWt → R² train: {r2_train:.3f}, R² test: {r2_test:.3f}")

plt.figure(figsize=(6,4))
plt.scatter(X_train, y_train, color='blue', label='Train')
plt.scatter(X_test, y_test, color='red', label='Test')
X_line = np.linspace(X.min(), X.max(), 100).reshape(-1,1)
plt.plot(X_line, model.predict(X_line), color='black', label='Linear fit')
plt.xlabel('MolWt')
plt.ylabel('Melting point (°C)')
plt.title(f'MolWt → R² train={r2_train:.2f}, R² test={r2_test:.2f}')
plt.legend()

Heavy Atoms Count

In [ ]:
heavyatom_values = data_clean["HeavyAtomCount"].values

In [ ]:
values_percentile_10 = np.percentile(heavyatom_values,10)
values_percentile_90 = np.percentile(heavyatom_values,90)

print(values_percentile_10)
print(values_percentile_90)

mask_10_to_90 = (heavyatom_values > values_percentile_10) & (heavyatom_values < values_percentile_90)

values_filtered = heavyatom_values[mask_10_to_90]
# the "~" is a boolean NOT operator
values_filtered_out = heavyatom_values[~mask_10_to_90]

#For regression (molwt + mpC together)
data_heavyatoms_filtered = data_clean[mask_10_to_90]

print("values.size:",heavyatom_values.size)
print("values_filtered.size",values_filtered.size)
print("values_filtered_out",values_filtered_out.size)

plt.plot(values_filtered,'.',label="filtered")
plt.plot(values_filtered_out,'.',label="filtered out")
plt.xlabel("Index")
plt.ylabel("Heavyatoms")
plt.legend()

In [ ]:
# make plots that shows the filtering

plt.hist(heavyatom_values,bins=100,label="Full set",density=True, alpha=0.5)
plt.hist(values_filtered,bins=50,label="Filtered set",density=True, alpha=0.5)
plt.xlabel("Molwt")
plt.ylabel("frequency")
# This is to plot vertical lines for the boundary at 10% and 90%
# using axvline. Here we will need to use the plt.gca() to get the
# axis object for the plot
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
#plt.show()
plt.legend()

# note that the KDE plot will "leak" outside the boundry
# lines as the KDE employs finite width Gaussian kernels
sns.kdeplot(heavyatom_values,label="Full set")
sns.kdeplot(values_filtered,label="Filtered set")
plt.xlabel("HeavyAtom")
plt.ylabel("density")
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
plt.legend()
#plt.show()

In [ ]:
# Linear regression
X = data_heavyatoms_filtered[['MolWt']].values
y = data_heavyatoms_filtered['mpC'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
r2_train = r2_score(y_train, model.predict(X_train))
r2_test = r2_score(y_test, model.predict(X_test))
print(f"Heavy Atoms → R² train: {r2_train:.3f}, R² test: {r2_test:.3f}")

plt.figure(figsize=(6,4))
plt.scatter(X_train, y_train, color='blue', label='Train')
plt.scatter(X_test, y_test, color='red', label='Test')
X_line = np.linspace(X.min(), X.max(), 100).reshape(-1,1)
plt.plot(X_line, model.predict(X_line), color='black', label='Linear fit')
plt.xlabel('HeavyAtoms')
plt.ylabel('Melting point (°C)')
plt.title(f'HeavyAtoms → R² train={r2_train:.2f}, R² test={r2_test:.2f}')
plt.legend()
plt.show ()

HBA

In [ ]:
hba_values = data_clean["HBA"].values
values_percentile_10 = np.percentile(hba_values,10)
values_percentile_90 = np.percentile(hba_values,90)

print(values_percentile_10)
print(values_percentile_90)

mask_10_to_90 = (hba_values > values_percentile_10) & (hba_values < values_percentile_90)

values_filtered = hba_values[mask_10_to_90]
# the "~" is a boolean NOT operator
values_filtered_out = hba_values[~mask_10_to_90]
#For regression (molwt + mpC together)
data_hba_filtered = data_clean[mask_10_to_90]

print("values.size:",hba_values.size)
print("values_filtered.size",values_filtered.size)
print("values_filtered_out",values_filtered_out.size)

plt.plot(values_filtered,'.',label="filtered")
plt.plot(values_filtered_out,'.',label="filtered out")
plt.xlabel("Index")
plt.ylabel("HBA")
plt.legend()

In [ ]:
# make plots that shows the filtering

plt.hist(hba_values,bins=100,label="Full set",density=True, alpha=0.5)
plt.hist(values_filtered,bins=50,label="Filtered set",density=True, alpha=0.5)
plt.xlabel("HBA")
plt.ylabel("frequency")
# This is to plot vertical lines for the boundary at 10% and 90%
# using axvline. Here we will need to use the plt.gca() to get the
# axis object for the plot
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
#plt.show()
plt.legend()

# note that the KDE plot will "leak" outside the boundry
# lines as the KDE employs finite width Gaussian kernels
sns.kdeplot(hba_values,label="Full set")
sns.kdeplot(values_filtered,label="Filtered set")
plt.xlabel("HBA")
plt.ylabel("density")
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
plt.legend()
#plt.show()

In [ ]:
# Linear regression
X = data_hba_filtered[['HBA']].values
y = data_hba_filtered['mpC'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
r2_train = r2_score(y_train, model.predict(X_train))
r2_test = r2_score(y_test, model.predict(X_test))
print(f"HBA → R² train: {r2_train:.3f}, R² test: {r2_test:.3f}")

plt.figure(figsize=(6,4))
plt.scatter(X_train, y_train, color='blue', label='Train')
plt.scatter(X_test, y_test, color='red', label='Test')
X_line = np.linspace(X.min(), X.max(), 100).reshape(-1,1)
plt.plot(X_line, model.predict(X_line), color='black', label='Linear fit')
plt.xlabel('HBA')
plt.ylabel('Melting point (°C)')
plt.title(f'HBA → R² train={r2_train:.2f}, R² test={r2_test:.2f}')
plt.legend()
plt.show()

HBD

In [ ]:
hbd_values = data_clean["HBD"].values
values_percentile_10 = np.percentile(hbd_values,10)
values_percentile_90 = np.percentile(hbd_values,90)

print(values_percentile_10)
print(values_percentile_90)

mask_10_to_90 = (hbd_values > values_percentile_10) & (hbd_values < values_percentile_90)

values_filtered = hbd_values[mask_10_to_90]
# the "~" is a boolean NOT operator
values_filtered_out = hbd_values[~mask_10_to_90]

#For regression (molwt + mpC together)
data_hbd_filtered = data_clean[mask_10_to_90]

print("values.size:",hbd_values.size)
print("values_filtered.size",values_filtered.size)
print("values_filtered_out",values_filtered_out.size)

plt.plot(values_filtered,'.',label="filtered")
plt.plot(values_filtered_out,'.',label="filtered out")
plt.xlabel("Index")
plt.ylabel("HBD")
plt.legend()



In [ ]:
# make plots that shows the filtering

plt.hist(hbd_values,bins=100,label="Full set",density=True, alpha=0.5)
plt.hist(values_filtered,bins=50,label="Filtered set",density=True, alpha=0.5)
plt.xlabel("HBD")
plt.ylabel("frequency")
# This is to plot vertical lines for the boundary at 10% and 90%
# using axvline. Here we will need to use the plt.gca() to get the
# axis object for the plot
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
#plt.show()
plt.legend()

# note that the KDE plot will "leak" outside the boundry
# lines as the KDE employs finite width Gaussian kernels
sns.kdeplot(hbd_values,label="Full set")
sns.kdeplot(values_filtered,label="Filtered set")
plt.xlabel("HBD")
plt.ylabel("density")
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
plt.legend()
#plt.show()

In [ ]:
# Linear regression
X = data_hbd_filtered[['HBD']].values
y = data_hbd_filtered['mpC'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
r2_train = r2_score(y_train, model.predict(X_train))
r2_test = r2_score(y_test, model.predict(X_test))
print(f"HBD → R² train: {r2_train:.3f}, R² test: {r2_test:.3f}")

plt.figure(figsize=(6,4))
plt.scatter(X_train, y_train, color='blue', label='Train')
plt.scatter(X_test, y_test, color='red', label='Test')
X_line = np.linspace(X.min(), X.max(), 100).reshape(-1,1)
plt.plot(X_line, model.predict(X_line), color='black', label='Linear fit')
plt.xlabel('HBD')
plt.ylabel('Melting point (°C)')
plt.title(f'HBD → R² train={r2_train:.2f}, R² test={r2_test:.2f}')
plt.legend()

LogP

In [ ]:
logp_values = data_clean["LogP"].values
values_percentile_10 = np.percentile(logp_values,10)
values_percentile_90 = np.percentile(logp_values,90)

print(values_percentile_10)
print(values_percentile_90)

mask_10_to_90 = (logp_values > values_percentile_10) & (logp_values < values_percentile_90)

values_filtered = logp_values[mask_10_to_90]
# the "~" is a boolean NOT operator
values_filtered_out = logp_values[~mask_10_to_90]

#For regression (molwt + mpC together)
data_logP_filtered = data_clean[mask_10_to_90]

print("values.size:",logp_values.size)
print("values_filtered.size",values_filtered.size)
print("values_filtered_out",values_filtered_out.size)

plt.plot(values_filtered,'.',label="filtered")
plt.plot(values_filtered_out,'.',label="filtered out")
plt.xlabel("Index")
plt.ylabel("LogP")
plt.legend()

In [ ]:
# make plots that shows the filtering

plt.hist(logp_values,bins=100,label="Full set",density=True, alpha=0.5)
plt.hist(values_filtered,bins=50,label="Filtered set",density=True, alpha=0.5)
plt.xlabel("LogP")
plt.ylabel("frequency")
# This is to plot vertical lines for the boundary at 10% and 90%
# using axvline. Here we will need to use the plt.gca() to get the
# axis object for the plot
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
#plt.show()
plt.legend()

# note that the KDE plot will "leak" outside the boundry
# lines as the KDE employs finite width Gaussian kernels
sns.kdeplot(logp_values,label="Full set")
sns.kdeplot(values_filtered,label="Filtered set")
plt.xlabel("LogP")
plt.ylabel("density")
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
plt.legend()
#plt.show()

In [ ]:
# Linear regression
X = data_logP_filtered[['LogP']].values
y = data_logP_filtered['mpC'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
r2_train = r2_score(y_train, model.predict(X_train))
r2_test = r2_score(y_test, model.predict(X_test))
print(f"LogP → R² train: {r2_train:.3f}, R² test: {r2_test:.3f}")

plt.figure(figsize=(6,4))
plt.scatter(X_train, y_train, color='blue', label='Train')
plt.scatter(X_test, y_test, color='red', label='Test')
X_line = np.linspace(X.min(), X.max(), 100).reshape(-1,1)
plt.plot(X_line, model.predict(X_line), color='black', label='Linear fit')
plt.xlabel('LogP')
plt.ylabel('Melting point (°C)')
plt.title(f'LogP → R² train={r2_train:.2f}, R² test={r2_test:.2f}')
plt.legend()

TPSA

In [ ]:
tpsa_values = data_clean["TPSA"].values
values_percentile_10 = np.percentile(tpsa_values,10)
values_percentile_90 = np.percentile(tpsa_values,90)

print(values_percentile_10)
print(values_percentile_90)

mask_10_to_90 = (tpsa_values > values_percentile_10) & (tpsa_values < values_percentile_90)

values_filtered = tpsa_values[mask_10_to_90]
# the "~" is a boolean NOT operator
values_filtered_out = tpsa_values[~mask_10_to_90]

#For regression (molwt + mpC together)
data_tpsa_filtered = data_clean[mask_10_to_90]

print("values.size:",tpsa_values.size)
print("values_filtered.size",values_filtered.size)
print("values_filtered_out",values_filtered_out.size)

plt.plot(values_filtered,'.',label="filtered")
plt.plot(values_filtered_out,'.',label="filtered out")
plt.xlabel("Index")
plt.ylabel("TPSA")
plt.legend()

In [ ]:
# make plots that shows the filtering

plt.hist(tpsa_values,bins=100,label="Full set",density=True, alpha=0.5)
plt.hist(values_filtered,bins=50,label="Filtered set",density=True, alpha=0.5)
plt.xlabel("TPSA")
plt.ylabel("frequency")
# This is to plot vertical lines for the boundary at 10% and 90%
# using axvline. Here we will need to use the plt.gca() to get the
# axis object for the plot
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
#plt.show()
plt.legend()

# note that the KDE plot will "leak" outside the boundry
# lines as the KDE employs finite width Gaussian kernels
sns.kdeplot(tpsa_values,label="Full set")
sns.kdeplot(values_filtered,label="Filtered set")
plt.xlabel("TPSA")
plt.ylabel("density")
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
plt.legend()

In [ ]:
# Linear regression
X = data_tpsa_filtered[['TPSA']].values
y = data_tpsa_filtered['mpC'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
r2_train = r2_score(y_train, model.predict(X_train))
r2_test = r2_score(y_test, model.predict(X_test))
print(f"TPSA → R² train: {r2_train:.3f}, R² test: {r2_test:.3f}")

plt.figure(figsize=(6,4))
plt.scatter(X_train, y_train, color='blue', label='Train')
plt.scatter(X_test, y_test, color='red', label='Test')
X_line = np.linspace(X.min(), X.max(), 100).reshape(-1,1)
plt.plot(X_line, model.predict(X_line), color='black', label='Linear fit')
plt.xlabel('TPSA')
plt.ylabel('Melting point (°C)')
plt.title(f'TPSA → R² train={r2_train:.2f}, R² test={r2_test:.2f}')
plt.legend()

TPSA-SP

In [ ]:
tpsa_sp_values = data_clean["TPSA_S_P"].values
values_percentile_10 = np.percentile(tpsa_sp_values,10)
values_percentile_90 = np.percentile(tpsa_sp_values,90)

print(values_percentile_10)
print(values_percentile_90)

mask_10_to_90 = (tpsa_sp_values > values_percentile_10) & (tpsa_sp_values < values_percentile_90)

values_filtered = tpsa_sp_values[mask_10_to_90]
# the "~" is a boolean NOT operator
values_filtered_out = tpsa_sp_values[~mask_10_to_90]

#For regression (molwt + mpC together)
data_tpsa_sp_filtered = data_clean[mask_10_to_90]

print("values.size:",tpsa_sp_values.size)
print("values_filtered.size",values_filtered.size)
print("values_filtered_out",values_filtered_out.size)

plt.plot(values_filtered,'.',label="filtered")
plt.plot(values_filtered_out,'.',label="filtered out")
plt.xlabel("Index")
plt.ylabel("TPSA_SP")
plt.legend()

In [ ]:
# make plots that shows the filtering

plt.hist(tpsa_sp_values,bins=100,label="Full set",density=True, alpha=0.5)
plt.hist(values_filtered,bins=50,label="Filtered set",density=True, alpha=0.5)
plt.xlabel("TPSA-SP")
plt.ylabel("frequency")
# This is to plot vertical lines for the boundary at 10% and 90%
# using axvline. Here we will need to use the plt.gca() to get the
# axis object for the plot
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
#plt.show()
plt.legend()

# note that the KDE plot will "leak" outside the boundry
# lines as the KDE employs finite width Gaussian kernels
sns.kdeplot(tpsa_sp_values,label="Full set")
sns.kdeplot(values_filtered,label="Filtered set")
plt.xlabel("TPSA-SP")
plt.ylabel("density")
ax = plt.gca()
ax.axvline(x=values_percentile_10, color='black', linestyle='--', linewidth=0.5)
ax.axvline(x=values_percentile_90, color='black', linestyle='--', linewidth=0.5)
plt.legend()

In [ ]:
# Linear regression
X = data_tpsa_sp_filtered[['TPSA_S_P']].values
y = data_tpsa_sp_filtered['mpC'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
r2_train = r2_score(y_train, model.predict(X_train))
r2_test = r2_score(y_test, model.predict(X_test))
print(f"TPSA_SP → R² train: {r2_train:.3f}, R² test: {r2_test:.3f}")

plt.figure(figsize=(6,4))
plt.scatter(X_train, y_train, color='blue', label='Train')
plt.scatter(X_test, y_test, color='red', label='Test')
X_line = np.linspace(X.min(), X.max(), 100).reshape(-1,1)
plt.plot(X_line, model.predict(X_line), color='black', label='Linear fit')
plt.xlabel('TPSA_SP')
plt.ylabel('Melting point (°C)')
plt.title(f'TPSA_SP → R² train={r2_train:.2f}, R² test={r2_test:.2f}')
plt.legend()

E) Performing RANSAC regression for molweight and TPSA .
These two properties correlate best with the melting point.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression, RANSACRegressor
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# X and y from  dataframe
X = data_clean[['MolWt']].values   # all data, no filtering
y = data_clean['mpC'].values

# RANSAC regression
ransac = RANSACRegressor(
    estimator=LinearRegression(),
    min_samples=0.5,
    random_state=42
)

ransac.fit(X, y)

# Identify inliers and outliers
inlier_mask = ransac.inlier_mask_
outlier_mask = ~inlier_mask

# Plot
plt.figure(figsize=(6,4))

plt.scatter(X[inlier_mask], y[inlier_mask],
            label='Inliers', alpha=0.7)

plt.scatter(X[outlier_mask], y[outlier_mask],
            label='Outliers', alpha=0.7)

X_line = np.linspace(X.min(), X.max(), 100).reshape(-1,1)
plt.plot(X_line, ransac.predict(X_line),
         color='black', linewidth=2, label='RANSAC fit')

plt.xlabel('MolWt')
plt.ylabel('Melting point (°C)')
plt.title('RANSAC Regression: MolWt vs Melting Point')
plt.legend()
plt.show()


In [ ]:


# X and y from  dataframe
X = data_clean[['TPSA']].values   # all data, no filtering
y = data_clean['mpC'].values

# RANSAC regression
ransac = RANSACRegressor(
    estimator=LinearRegression(),
    min_samples=0.5,
    random_state=42
)

ransac.fit(X, y)

# Identify inliers and outliers
inlier_mask = ransac.inlier_mask_
outlier_mask = ~inlier_mask

# Plot
plt.figure(figsize=(6,4))

plt.scatter(X[inlier_mask], y[inlier_mask],
            label='Inliers', alpha=0.7)

plt.scatter(X[outlier_mask], y[outlier_mask],
            label='Outliers', alpha=0.7)

X_line = np.linspace(X.min(), X.max(), 100).reshape(-1,1)
plt.plot(X_line, ransac.predict(X_line),
         color='black', linewidth=2, label='RANSAC fit')

plt.xlabel('TPSA')
plt.ylabel('Melting point (°C)')
plt.title('RANSAC Regression: TPSA vs Melting Point')
plt.legend()
plt.show()


### Task 2 - Optional 5 points

Here we will consider a dataset of two variables $x$ and $y$ sampled from a two-dimensional probability density $P(x,y)$ that is unknown.

The dataset is given as a time series in the file `Dataset_RotatedWQ-Potential.data`.

The main task is to perform a Gaussian Mixture Model analysis on this two-dimensional dataset.

#### A)
Plot the dataset, both the time series and also a scatter plot for the $x$ and $y$ variables.

Looking at the scatter plot, how many Gaussian components do you think are needed in the Gaussian Mixture Model analysis?

#### B)
Using Seaborn (or scikit-learn) estimate the two-dimensional probability density $P(x,y)$ using kernel density estimation.

#### C)
Perform a Gaussian Mixture Model analysis for a different number of components, and obtain the Bayesian information criterion (bic) and Akaike information criterion (aic) values and based on them identify the optimal number of components (remember that for both a lower value is better).

#### D)
For the optimal number of components, perform a final Gaussian Mixture Model analysis that you will analyze.

- What is the weight of each Gaussian components.

- What is the percentage of samples that are hard classifed to each cluster.

- Make a scatter plot that shows how the samples are hard classifed to each cluster. In this plot, indicate the center of each Gaussian components.

- Make figures that shows how the samples are soft classifed to each cluster (e.g., the probablity that they belong to a given cluster). In each plot, indicate the center of corresponding Gaussian components.

- Plot a two-dimensional surface of the $P(x,y)$ estimated by the Gaussian Mixture Model. How does this compare to the KDE plot from B)?


In [ ]:
# Bash script to download all the dataset. Don't worry if you don't understand it
%%bash

url="https://raw.githubusercontent.com/valsson-group/UNT-ChemicalApplicationsOfMachineLearning-Spring2026/refs/heads/main/Assignment-2/"
dataset_filename="Dataset_RotatedWQ-Potential.data"

rm -f ${dataset_filename}

wget ${url}/${dataset_filename} &> /dev/null

ls



A) Load data + time series + scatter plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
# If comma-separated, change delimiter=','
data = np.loadtxt("Dataset_RotatedWQ-Potential.data")

x = data[:, 0]
y = data[:, 1]

# Time series plot
plt.figure(figsize=(10,4))
plt.plot(x, label='x')
plt.plot(y, label='y')
plt.xlabel("Time index")
plt.ylabel("Value")
plt.title("Time series of x and y")
plt.legend()
plt.show()

# Scatter plot
plt.figure(figsize=(6,6))
plt.scatter(x, y, s=5, alpha=0.5)
plt.xlabel("x")
plt.ylabel("y")
plt.title("Scatter plot of x vs y")
plt.show()


B) 2D KDE estimate of P(x,y)

In [ ]:
plt.figure(figsize=(6,6))
sns.kdeplot(
    x=x, y=y,
    fill=True,
    cmap="viridis",
    levels=50
)
plt.xlabel("x")
plt.ylabel("y")
plt.title("2D KDE estimate of P(x, y)")
plt.show()


C) GMM with different components + BIC/AIC

In [ ]:
from sklearn.mixture import GaussianMixture

X = np.column_stack((x, y))

n_components_range = range(1, 7)
bic = []
aic = []

for n in n_components_range:
    gmm = GaussianMixture(
        n_components=n,
        covariance_type='full',
        random_state=42
    )
    gmm.fit(X)
    bic.append(gmm.bic(X))
    aic.append(gmm.aic(X))

# Plot BIC/AIC
plt.figure(figsize=(6,4))
plt.plot(n_components_range, bic, marker='o', label='BIC')
plt.plot(n_components_range, aic, marker='o', label='AIC')
plt.xlabel("Number of components")
plt.ylabel("Criterion value")
plt.title("Model selection using BIC and AIC")
plt.legend()
plt.show()


In [ ]:
N_OPT = 3

gmm = GaussianMixture(
    n_components=N_OPT,
    covariance_type='full',
    random_state=42
)
gmm.fit(X)

labels = gmm.predict(X)
probs = gmm.predict_proba(X)
means = gmm.means_
weights = gmm.weights_


D1) Weight of each Gaussian component

In [ ]:
for i, w in enumerate(weights):
    print(f"Component {i}: weight = {w:.3f}")


D2) Percentage of samples hard-classified to each cluster

In [ ]:
for i in range(N_OPT):
    perc = np.mean(labels == i) * 100
    print(f"Component {i}: {perc:.2f}% of samples")


D3) Scatter plot — hard classification + centers

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(x, y, c=labels, cmap='tab10', s=5)
plt.scatter(means[:,0], means[:,1],
            c='black', marker='x', s=100, label='Centers')
plt.xlabel("x")
plt.ylabel("y")
plt.title("Hard classification by GMM")
plt.legend()
plt.show()


D4) Soft classification plots (probabilities)

In [ ]:
for i in range(N_OPT):
    plt.figure(figsize=(6,6))
    plt.scatter(x, y, c=probs[:, i], cmap='viridis', s=5)
    plt.colorbar(label=f"P(cluster {i})")
    plt.scatter(means[i,0], means[i,1],
                c='red', marker='x', s=100)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(f"Soft classification: cluster {i}")
    plt.show()


D5) 2D surface of
𝑃
(
𝑥
,
𝑦
)
P(x,y) from GMM

In [ ]:
# Grid
x_grid = np.linspace(x.min(), x.max(), 100)
y_grid = np.linspace(y.min(), y.max(), 100)
Xg, Yg = np.meshgrid(x_grid, y_grid)
XY = np.column_stack([Xg.ravel(), Yg.ravel()])

Z = np.exp(gmm.score_samples(XY))
Z = Z.reshape(Xg.shape)

# Surface / contour plot
plt.figure(figsize=(6,6))
plt.contourf(Xg, Yg, Z, levels=50, cmap='viridis')
plt.colorbar(label='P(x,y)')
plt.scatter(means[:,0], means[:,1],
            c='red', marker='x', s=100)
plt.xlabel("x")
plt.ylabel("y")
plt.title("GMM estimate of P(x, y)")
plt.show()
